In [ ]:
'''code to read a set of logfiles and find information of runtime, memory, # of CPU/GPUs, etc.
 and write to a new document for the purpose of analysis of computing power and speed
 @author:wuaudrey'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install ase
import ase
from ase.io import read, write
import re

In [ ]:
'''GPU log files are slightly different, and this code will eventually be updated to account for that,
but currently this should only be used for CPU log files'''

machine_name = 'nalen'  # update these lines!
logfile_names = ['trial_sim.log',
                 'trial_sim_2.log'
                 ]


time = 'Total wall time:'
memory = "CPU use with"

def find_data(filename):
    # read all lines from the file
    with open(filename, "r") as f:
        lines = f.readlines()

    # find data
        for i, line in enumerate(lines):
            if time in line:
                wall_time = lines[i].strip().split("Total wall time:")[1].strip()
            if memory in line:
              match = re.search(r'(\d+)\s+MPI tasks\s+x\s+(\d+)\s+OpenMP threads', line)
              if match:
                    mpi_tasks = match.group(1)
                    threads = match.group(2)

    return mpi_tasks, threads, wall_time

# write data to a new file
open("trial_sim_data.txt", 'x')
with open("trial_sim_data.txt", "w") as f:
    f.write(f"#{machine_name}")
    f.write(f"#{'CPUs'} \t {'Threads'} \t {'Wall Time'} \n")
    for filename in logfile_names:
        mpi_tasks, threads, wall_time = find_data(filename)
        f.write(f"{mpi_tasks} \t {threads} \t {wall_time} \n")